# portfolio

In [1]:
import vectorbt as vbt

from vectorbt.portfolio.enums import SizeType

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from numba import njit, f8, i8, b1, optional
from datetime import datetime

In [3]:
price = pd.Series([1, 2, 3, 2, 1], index=pd.Index([
    datetime(2020, 1, 1),
    datetime(2020, 1, 2),
    datetime(2020, 1, 3),
    datetime(2020, 1, 4),
    datetime(2020, 1, 5)
]))

print(price)

2020-01-01    1
2020-01-02    2
2020-01-03    3
2020-01-04    2
2020-01-05    1
dtype: int64


In [4]:
big_price = pd.Series(np.random.uniform(size=(1000,)))
big_price.index = [datetime(2018, 1, 1) + timedelta(days=i) for i in range(1000)]

print(big_price.shape)

(1000,)


In [5]:
entries = pd.DataFrame({
    'a': [True, True, True, False, False],
    'b': [True, False, True, False, True],
    'c': [False, True, False, True, False]
}, index=price.index)
exits = pd.DataFrame({
    'a': [False, False, True, True, True],
    'b': [False, True, False, True, False],
    'c': [True, False, True, False, True]
}, index=price.index)

print(entries)
print(exits)

                a      b      c
2020-01-01   True   True  False
2020-01-02   True  False   True
2020-01-03   True   True  False
2020-01-04  False  False   True
2020-01-05  False   True  False
                a      b      c
2020-01-01  False  False   True
2020-01-02  False   True  False
2020-01-03   True  False   True
2020-01-04   True   True  False
2020-01-05   True  False   True


In [6]:
big_entries, big_exits = pd.DataFrame.vbt.signals.generate_random_entries_and_exits(
    (1000, 1000), 100, seed=42, index=big_price.index)

print(big_entries.shape)
print(big_exits.shape)

(1000, 1000)
(1000, 1000)


In [7]:
# Disable caching for performance testing
# NOTE: Expect waterfall of executions, since some attributes depend on other attributes 
# that have to be calculated again and again
vbt.defaults.caching = False

## from_signals

In [8]:
portfolio = vbt.Portfolio.from_signals(price, entries['a'], exits['a'])
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_signals(big_price, big_entries.iloc[:, 0], big_exits.iloc[:, 0])

   col  idx   size  price  fees  side
0    0    0  100.0    1.0   0.0     0
1    0    3  100.0    2.0   0.0     1
2020-01-01    100.0
2020-01-02    100.0
2020-01-03    100.0
2020-01-04      0.0
2020-01-05      0.0
Name: a, dtype: float64
2020-01-01      0.0
2020-01-02      0.0
2020-01-03      0.0
2020-01-04    200.0
2020-01-05    200.0
Name: a, dtype: float64
4.77 ms ± 255 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
portfolio = vbt.Portfolio.from_signals(price, entries, exits, size=1.)
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_signals(big_price, big_entries, big_exits, size=1.)

    col  idx  size  price  fees  side
0     0    0   1.0    1.0   0.0     0
1     0    3   1.0    2.0   0.0     1
2     1    0   1.0    1.0   0.0     0
3     1    1   1.0    2.0   0.0     1
4     1    2   1.0    3.0   0.0     0
5     1    3   1.0    2.0   0.0     1
6     1    4   1.0    1.0   0.0     0
7     2    1   1.0    2.0   0.0     0
8     2    2   1.0    3.0   0.0     1
9     2    3   1.0    2.0   0.0     0
10    2    4   1.0    1.0   0.0     1
              a    b    c
2020-01-01  1.0  1.0  0.0
2020-01-02  1.0  0.0  1.0
2020-01-03  1.0  1.0  0.0
2020-01-04  0.0  0.0  1.0
2020-01-05  0.0  1.0  0.0
                a      b      c
2020-01-01   99.0   99.0  100.0
2020-01-02   99.0  101.0   98.0
2020-01-03   99.0   98.0  101.0
2020-01-04  101.0  100.0   99.0
2020-01-05  101.0   99.0  100.0
37.8 ms ± 8.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
portfolio = vbt.Portfolio.from_signals(price, entries, exits, size=np.inf) # all in/out
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_signals(big_price, big_entries, big_exits, size=np.inf)

    col  idx        size  price  fees  side
0     0    0  100.000000    1.0   0.0     0
1     0    3  100.000000    2.0   0.0     1
2     1    0  100.000000    1.0   0.0     0
3     1    1  100.000000    2.0   0.0     1
4     1    2   66.666667    3.0   0.0     0
5     1    3   66.666667    2.0   0.0     1
6     1    4  133.333333    1.0   0.0     0
7     2    1   50.000000    2.0   0.0     0
8     2    2   50.000000    3.0   0.0     1
9     2    3   75.000000    2.0   0.0     0
10    2    4   75.000000    1.0   0.0     1
                a           b     c
2020-01-01  100.0  100.000000   0.0
2020-01-02  100.0    0.000000  50.0
2020-01-03  100.0   66.666667   0.0
2020-01-04    0.0    0.000000  75.0
2020-01-05    0.0  133.333333   0.0
                a           b      c
2020-01-01    0.0    0.000000  100.0
2020-01-02    0.0  200.000000    0.0
2020-01-03    0.0    0.000000  150.0
2020-01-04  200.0  133.333333    0.0
2020-01-05  200.0    0.000000   75.0
41.5 ms ± 5.95 ms per loop (mean ±

In [11]:
portfolio = vbt.Portfolio.from_signals(price, entries, exits, size=1, fees=0.01) # w/ fees
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_signals(big_price, big_entries, big_exits, size=1, fees=0.01)

    col  idx  size  price  fees  side
0     0    0   1.0    1.0  0.01     0
1     0    3   1.0    2.0  0.02     1
2     1    0   1.0    1.0  0.01     0
3     1    1   1.0    2.0  0.02     1
4     1    2   1.0    3.0  0.03     0
5     1    3   1.0    2.0  0.02     1
6     1    4   1.0    1.0  0.01     0
7     2    1   1.0    2.0  0.02     0
8     2    2   1.0    3.0  0.03     1
9     2    3   1.0    2.0  0.02     0
10    2    4   1.0    1.0  0.01     1
              a    b    c
2020-01-01  1.0  1.0  0.0
2020-01-02  1.0  0.0  1.0
2020-01-03  1.0  1.0  0.0
2020-01-04  0.0  0.0  1.0
2020-01-05  0.0  1.0  0.0
                 a       b       c
2020-01-01   98.99   98.99  100.00
2020-01-02   98.99  100.97   97.98
2020-01-03   98.99   97.94  100.95
2020-01-04  100.97   99.92   98.93
2020-01-05  100.97   98.91   99.92
41.7 ms ± 2.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
portfolio = vbt.Portfolio.from_signals(price, entries, exits, size=1, fixed_fees=1) # w/ fixed fees
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_signals(big_price, big_entries, big_exits, size=1, fixed_fees=1)

    col  idx  size  price  fees  side
0     0    0   1.0    1.0   1.0     0
1     0    3   1.0    2.0   1.0     1
2     1    0   1.0    1.0   1.0     0
3     1    1   1.0    2.0   1.0     1
4     1    2   1.0    3.0   1.0     0
5     1    3   1.0    2.0   1.0     1
6     1    4   1.0    1.0   1.0     0
7     2    1   1.0    2.0   1.0     0
8     2    2   1.0    3.0   1.0     1
9     2    3   1.0    2.0   1.0     0
10    2    4   1.0    1.0   1.0     1
              a    b    c
2020-01-01  1.0  1.0  0.0
2020-01-02  1.0  0.0  1.0
2020-01-03  1.0  1.0  0.0
2020-01-04  0.0  0.0  1.0
2020-01-05  0.0  1.0  0.0
               a     b      c
2020-01-01  98.0  98.0  100.0
2020-01-02  98.0  99.0   97.0
2020-01-03  98.0  95.0   99.0
2020-01-04  99.0  96.0   96.0
2020-01-05  99.0  94.0   96.0
39.1 ms ± 2.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
portfolio = vbt.Portfolio.from_signals(price, entries, exits, size=1, slippage=0.01) # w/ slippage
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_signals(big_price, big_entries, big_exits, size=1, slippage=0.01)

    col  idx  size  price  fees  side
0     0    0   1.0   1.01   0.0     0
1     0    3   1.0   1.98   0.0     1
2     1    0   1.0   1.01   0.0     0
3     1    1   1.0   1.98   0.0     1
4     1    2   1.0   3.03   0.0     0
5     1    3   1.0   1.98   0.0     1
6     1    4   1.0   1.01   0.0     0
7     2    1   1.0   2.02   0.0     0
8     2    2   1.0   2.97   0.0     1
9     2    3   1.0   2.02   0.0     0
10    2    4   1.0   0.99   0.0     1
              a    b    c
2020-01-01  1.0  1.0  0.0
2020-01-02  1.0  0.0  1.0
2020-01-03  1.0  1.0  0.0
2020-01-04  0.0  0.0  1.0
2020-01-05  0.0  1.0  0.0
                 a       b       c
2020-01-01   98.99   98.99  100.00
2020-01-02   98.99  100.97   97.98
2020-01-03   98.99   97.94  100.95
2020-01-04  100.97   99.92   98.93
2020-01-05  100.97   98.91   99.92
34.7 ms ± 2.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
# entry & exit price
portfolio = vbt.Portfolio.from_signals(
    price, entries, exits, size=1, entry_price=price*0.9, exit_price=price*1.1)
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_signals(\
    big_price, big_entries, big_exits, size=1, entry_price=big_price*0.9, exit_price=big_price*1.1)

    col  idx  size  price  fees  side
0     0    0   1.0    0.9   0.0     0
1     0    3   1.0    2.2   0.0     1
2     1    0   1.0    0.9   0.0     0
3     1    1   1.0    2.2   0.0     1
4     1    2   1.0    2.7   0.0     0
5     1    3   1.0    2.2   0.0     1
6     1    4   1.0    0.9   0.0     0
7     2    1   1.0    1.8   0.0     0
8     2    2   1.0    3.3   0.0     1
9     2    3   1.0    1.8   0.0     0
10    2    4   1.0    1.1   0.0     1
              a    b    c
2020-01-01  1.0  1.0  0.0
2020-01-02  1.0  0.0  1.0
2020-01-03  1.0  1.0  0.0
2020-01-04  0.0  0.0  1.0
2020-01-05  0.0  1.0  0.0
                a      b      c
2020-01-01   99.1   99.1  100.0
2020-01-02   99.1  101.3   98.2
2020-01-03   99.1   98.6  101.5
2020-01-04  101.3  100.8   99.7
2020-01-05  101.3   99.9  100.8
35.6 ms ± 2.86 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
portfolio = vbt.Portfolio.from_signals(price, entries, exits, size=1, accumulate=True) # w/ accumulation
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_signals(big_price, big_entries, big_exits, size=1, accumulate=True)

    col  idx  size  price  fees  side
0     0    0   1.0    1.0   0.0     0
1     0    1   1.0    2.0   0.0     0
2     0    2   1.0    3.0   0.0     1
3     0    3   1.0    2.0   0.0     1
4     1    0   1.0    1.0   0.0     0
5     1    1   1.0    2.0   0.0     1
6     1    2   1.0    3.0   0.0     0
7     1    3   1.0    2.0   0.0     1
8     1    4   1.0    1.0   0.0     0
9     2    1   1.0    2.0   0.0     0
10    2    2   1.0    3.0   0.0     1
11    2    3   1.0    2.0   0.0     0
12    2    4   1.0    1.0   0.0     1
              a    b    c
2020-01-01  1.0  1.0  0.0
2020-01-02  2.0  0.0  1.0
2020-01-03  1.0  1.0  0.0
2020-01-04  0.0  0.0  1.0
2020-01-05  0.0  1.0  0.0
                a      b      c
2020-01-01   99.0   99.0  100.0
2020-01-02   97.0  101.0   98.0
2020-01-03  100.0   98.0  101.0
2020-01-04  102.0  100.0   99.0
2020-01-05  102.0   99.0  100.0
34.9 ms ± 2.81 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
portfolio = vbt.Portfolio.from_signals(
    price, 
    entries=[True, False, True, True, True],
    exits=[False, True, False, False, True],
    size=[1, 2, 3, 4, 5], 
    size_type=SizeType.Shares, accumulate=False)
print(portfolio.shares)

portfolio = vbt.Portfolio.from_signals(
    price, 
    entries=[True, False, True, True, True],
    exits=[False, True, False, False, True],
    size=[1, 2, 3, 4, 5], 
    size_type=SizeType.Cash, accumulate=False)
print(portfolio.cash)

portfolio = vbt.Portfolio.from_signals(
    price, 
    entries=[True, False, True, True, True],
    exits=[False, True, False, False, True],
    size=[1, 2, 3, 4, 5], 
    size_type=SizeType.Shares, accumulate=True)
print(portfolio.shares)

portfolio = vbt.Portfolio.from_signals(
    price, 
    entries=[True, False, True, True, True],
    exits=[False, True, False, False, True],
    size=[1, 2, 3, 4, 5], 
    size_type=SizeType.Cash, accumulate=True)
print(portfolio.cash)

2020-01-01    1.0
2020-01-02    0.0
2020-01-03    3.0
2020-01-04    3.0
2020-01-05    5.0
dtype: float64
2020-01-01     99.0
2020-01-02    101.0
2020-01-03     98.0
2020-01-04     98.0
2020-01-05      5.0
dtype: float64
2020-01-01    1.0
2020-01-02    0.0
2020-01-03    3.0
2020-01-04    7.0
2020-01-05    5.0
dtype: float64
2020-01-01     99.0
2020-01-02    101.0
2020-01-03     98.0
2020-01-04     94.0
2020-01-05      5.0
dtype: float64


## from_orders

In [17]:
order_size = pd.DataFrame({
    'a': [1, 0.1, -1, -0.1, -1],
    'b': [1, 1, 1, 1, -np.inf],
    'c': [np.inf, -np.inf, np.inf, -np.inf, np.inf]
}, index=price.index)
big_order_size = pd.DataFrame.vbt.empty((1000, 1000), 1)

In [18]:
portfolio = vbt.Portfolio.from_orders(price, order_size['a'])
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_orders(big_price, big_order_size.iloc[:, 0])

   col  idx  size  price  fees  side
0    0    0   1.0    1.0   0.0     0
1    0    1   0.1    2.0   0.0     0
2    0    2   1.0    3.0   0.0     1
3    0    3   0.1    2.0   0.0     1
2020-01-01    1.0
2020-01-02    1.1
2020-01-03    0.1
2020-01-04    0.0
2020-01-05    0.0
Name: a, dtype: float64
2020-01-01     99.0
2020-01-02     98.8
2020-01-03    101.8
2020-01-04    102.0
2020-01-05    102.0
Name: a, dtype: float64
4 ms ± 50.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
portfolio = vbt.Portfolio.from_orders(price, order_size)
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_orders(big_price, big_order_size)

    col  idx        size  price  fees  side
0     0    0    1.000000    1.0   0.0     0
1     0    1    0.100000    2.0   0.0     0
2     0    2    1.000000    3.0   0.0     1
3     0    3    0.100000    2.0   0.0     1
4     1    0    1.000000    1.0   0.0     0
5     1    1    1.000000    2.0   0.0     0
6     1    2    1.000000    3.0   0.0     0
7     1    3    1.000000    2.0   0.0     0
8     1    4    4.000000    1.0   0.0     1
9     2    0  100.000000    1.0   0.0     0
10    2    1  100.000000    2.0   0.0     1
11    2    2   66.666667    3.0   0.0     0
12    2    3   66.666667    2.0   0.0     1
13    2    4  133.333333    1.0   0.0     0
              a    b           c
2020-01-01  1.0  1.0  100.000000
2020-01-02  1.1  2.0    0.000000
2020-01-03  0.1  3.0   66.666667
2020-01-04  0.0  4.0    0.000000
2020-01-05  0.0  0.0  133.333333
                a     b           c
2020-01-01   99.0  99.0    0.000000
2020-01-02   98.8  97.0  200.000000
2020-01-03  101.8  94.0    0.00000

In [20]:
portfolio = vbt.Portfolio.from_orders(price, [1, 2, 3, 4, 5], size_type=SizeType.Shares)
print(portfolio.shares)

portfolio = vbt.Portfolio.from_orders(price, [1, 2, 3, 4, 5], size_type=SizeType.TargetShares)
print(portfolio.shares)

portfolio = vbt.Portfolio.from_orders(price, [1, 2, 3, 4, 5], size_type=SizeType.Cash)
print(portfolio.cash)

portfolio = vbt.Portfolio.from_orders(price, [1, 2, 3, 4, 5], size_type=SizeType.TargetCash)
print(portfolio.cash)

portfolio = vbt.Portfolio.from_orders(price, [1, 2, 3, 4, 5], size_type=SizeType.TargetValue)
print(portfolio.shares)

portfolio = vbt.Portfolio.from_orders(price, [0.1, 0.2, 0.3, 0.4, 0.5], size_type=SizeType.TargetPercent)
print(portfolio.shares)

2020-01-01     1.0
2020-01-02     3.0
2020-01-03     6.0
2020-01-04    10.0
2020-01-05    15.0
dtype: float64
2020-01-01    1.0
2020-01-02    2.0
2020-01-03    3.0
2020-01-04    4.0
2020-01-05    5.0
dtype: float64
2020-01-01    99.0
2020-01-02    97.0
2020-01-03    94.0
2020-01-04    90.0
2020-01-05    85.0
dtype: float64
2020-01-01    1.0
2020-01-02    2.0
2020-01-03    3.0
2020-01-04    4.0
2020-01-05    5.0
dtype: float64
2020-01-01    1.0
2020-01-02    1.0
2020-01-03    1.0
2020-01-04    2.0
2020-01-05    5.0
dtype: float64
2020-01-01    10.00
2020-01-02    11.00
2020-01-03    12.10
2020-01-04    21.78
2020-01-05    43.56
dtype: float64


## from_order_func

In [21]:
@njit
def order_func_nb(order_context, price, fees, fixed_fees, slippage):
    i = order_context.i
    col = order_context.col
    size = col + 1
    if i % 2 == 1:
        size *= -1
    return vbt.portfolio.nb.Order(
        size, SizeType.Shares, price[i, col], fees[i, col], fixed_fees[i, col], slippage[i, col])

In [22]:
portfolio = vbt.Portfolio.from_order_func(
    price, 
    order_func_nb, 
    price.values[:, None],
    np.full(price.shape, 0.01)[:, None],
    np.full(price.shape, 1)[:, None],
    np.full(price.shape, 0.01)[:, None]
)
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_order_func(\
    big_price, \
    order_func_nb, \
    big_price.values[:, None],\
    np.full(big_price.shape, 0.01)[:, None],\
    np.full(big_price.shape, 1)[:, None],\
    np.full(big_price.shape, 0.01)[:, None]\
)

   col  idx  size  price    fees  side
0    0    0   1.0   1.01  1.0101     0
1    0    1   1.0   1.98  1.0198     1
2    0    2   1.0   3.03  1.0303     0
3    0    3   1.0   1.98  1.0198     1
4    0    4   1.0   1.01  1.0101     0
2020-01-01    1.0
2020-01-02    0.0
2020-01-03    1.0
2020-01-04    0.0
2020-01-05    1.0
dtype: float64
2020-01-01    97.9799
2020-01-02    98.9401
2020-01-03    94.8798
2020-01-04    95.8400
2020-01-05    93.8199
dtype: float64
3.53 ms ± 366 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
portfolio = vbt.Portfolio.from_order_func(
    price.vbt.tile(3, keys=['a', 'b', 'c']), 
    order_func_nb, 
    price.vbt.tile(3).values,
    np.full((price.shape[0], 3), 0.01),
    np.full((price.shape[0], 3), 1),
    np.full((price.shape[0], 3), 0.01)
)
print(portfolio.orders.records)
print(portfolio.shares)
print(portfolio.cash)

%timeit vbt.Portfolio.from_order_func(\
    big_price.vbt.tile(1000), \
    order_func_nb, \
    big_price.vbt.tile(1000).values,\
    np.full((big_price.shape[0], 1000), 0.01),\
    np.full((big_price.shape[0], 1000), 1),\
    np.full((big_price.shape[0], 1000), 0.01)\
)

    col  idx  size  price    fees  side
0     0    0   1.0   1.01  1.0101     0
1     0    1   1.0   1.98  1.0198     1
2     0    2   1.0   3.03  1.0303     0
3     0    3   1.0   1.98  1.0198     1
4     0    4   1.0   1.01  1.0101     0
5     1    0   2.0   1.01  1.0202     0
6     1    1   2.0   1.98  1.0396     1
7     1    2   2.0   3.03  1.0606     0
8     1    3   2.0   1.98  1.0396     1
9     1    4   2.0   1.01  1.0202     0
10    2    0   3.0   1.01  1.0303     0
11    2    1   3.0   1.98  1.0594     1
12    2    2   3.0   3.03  1.0909     0
13    2    3   3.0   1.98  1.0594     1
14    2    4   3.0   1.01  1.0303     0
              a    b    c
2020-01-01  1.0  2.0  3.0
2020-01-02  0.0  0.0  0.0
2020-01-03  1.0  2.0  3.0
2020-01-04  0.0  0.0  0.0
2020-01-05  1.0  2.0  3.0
                  a        b         c
2020-01-01  97.9799  96.9598   95.9397
2020-01-02  98.9401  99.8802  100.8203
2020-01-03  94.8798  92.7596   90.6394
2020-01-04  95.8400  95.6800   95.5200
2020-01-0

## Indexing

In [24]:
init_capital = [100, 200, 300]
levy_alpha = [1., 2., 3.]
risk_free = [0.01, 0.02, 0.03]
required_return = [0.1, 0.2, 0.3]
cutoff = [0.01, 0.02, 0.03]
factor_returns = price.vbt.combine_with_multiple(
    [0.9, 1., 1.1],
    combine_func=np.multiply,
    concat=True,
    keys=entries.columns
)
portfolio = vbt.Portfolio.from_signals(
    price, entries, exits,
    fees=0.01,
    init_capital=init_capital,
    freq='1 days',
    year_freq='252 days',
    levy_alpha=levy_alpha,
    risk_free=risk_free,
    required_return=required_return,
    cutoff=cutoff,
    factor_returns=factor_returns
)
print(portfolio.wrapper.shape)

big_portfolio = vbt.Portfolio.from_signals(
    big_price, big_entries, big_exits, fees=0.01, freq='1 day', year_freq='252 days',
    levy_alpha=levy_alpha[0], risk_free=risk_free[0], required_return=required_return[0], 
    cutoff=cutoff[0], factor_returns=big_price)
print(big_portfolio.wrapper.shape)

(5, 3)
(1000, 1000)


In [25]:
print(portfolio.main_price)
print(portfolio.init_capital)
print(portfolio.orders.records)
print(portfolio.cash)
print(portfolio.shares)
print(portfolio.freq)
print(portfolio.year_freq)
print(portfolio.levy_alpha)
print(portfolio.risk_free)
print(portfolio.required_return)
print(portfolio.cutoff)
print(portfolio.factor_returns)

            a  b  c
2020-01-01  1  1  1
2020-01-02  2  2  2
2020-01-03  3  3  3
2020-01-04  2  2  2
2020-01-05  1  1  1
a    100
b    200
c    300
dtype: int64
    col  idx        size  price      fees  side
0     0    0   99.009901    1.0  0.990099     0
1     0    3   99.009901    2.0  1.980198     1
2     1    0  198.019802    1.0  1.980198     0
3     1    1  198.019802    2.0  3.960396     1
4     1    2  129.399079    3.0  3.881972     0
5     1    3  129.399079    2.0  2.587982     1
6     1    4  253.673441    1.0  2.536734     0
7     2    1  148.514851    2.0  2.970297     0
8     2    2  148.514851    3.0  4.455446     1
9     2    3  218.360945    2.0  4.367219     0
10    2    4  218.360945    1.0  2.183609     1
                     a           b           c
2020-01-01    0.000000    0.000000  300.000000
2020-01-02    0.000000  392.079208    0.000000
2020-01-03    0.000000    0.000000  441.089109
2020-01-04  196.039604  256.210175    0.000000
2020-01-05  196.039604    0.0

In [26]:
print(portfolio['a'].main_price)
print(portfolio['a'].init_capital)
print(portfolio['a'].orders.records)
print(portfolio['a'].cash)
print(portfolio['a'].shares)
print(portfolio['a'].freq)
print(portfolio['a'].year_freq)
print(portfolio['a'].levy_alpha)
print(portfolio['a'].risk_free)
print(portfolio['a'].required_return)
print(portfolio['a'].cutoff)
print(portfolio['a'].factor_returns)

2020-01-01    1
2020-01-02    2
2020-01-03    3
2020-01-04    2
2020-01-05    1
Name: a, dtype: int64
100
   col  idx       size  price      fees  side
0    0    0  99.009901    1.0  0.990099     0
1    0    3  99.009901    2.0  1.980198     1
2020-01-01      0.000000
2020-01-02      0.000000
2020-01-03      0.000000
2020-01-04    196.039604
2020-01-05    196.039604
Name: a, dtype: float64
2020-01-01    99.009901
2020-01-02    99.009901
2020-01-03    99.009901
2020-01-04     0.000000
2020-01-05     0.000000
Name: a, dtype: float64
1 days 00:00:00
252 days 00:00:00
1.0
0.01
0.1
0.01
2020-01-01    0.9
2020-01-02    1.8
2020-01-03    2.7
2020-01-04    1.8
2020-01-05    0.9
Name: a, dtype: float64


In [27]:
print(portfolio.main_price.iloc[:, 0])
%timeit big_portfolio.main_price.iloc[:, 0]

print(portfolio.iloc[:, 0].main_price)
%timeit big_portfolio.iloc[:, 0].main_price # slower since requires does much more processing

2020-01-01    1
2020-01-02    2
2020-01-03    3
2020-01-04    2
2020-01-05    1
Name: a, dtype: int64
64.3 µs ± 1.51 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
2020-01-01    1
2020-01-02    2
2020-01-03    3
2020-01-04    2
2020-01-05    1
Name: a, dtype: int64
4.98 ms ± 365 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
print(portfolio.main_price.iloc[:, np.arange(portfolio.main_price.shape[1])])
%timeit big_portfolio.main_price.iloc[:, np.arange(big_portfolio.main_price.shape[1])]

print(portfolio.iloc[:, np.arange(portfolio.main_price.shape[1])].main_price)
%timeit big_portfolio.iloc[:, np.arange(big_portfolio.main_price.shape[1])].main_price

            a  b  c
2020-01-01  1  1  1
2020-01-02  2  2  2
2020-01-03  3  3  3
2020-01-04  2  2  2
2020-01-05  1  1  1
3.42 ms ± 294 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
            a  b  c
2020-01-01  1  1  1
2020-01-02  2  2  2
2020-01-03  3  3  3
2020-01-04  2  2  2
2020-01-05  1  1  1
29.8 ms ± 2.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
orders = portfolio.orders
big_orders = big_portfolio.orders

print(orders.iloc[:, 0].records)
%timeit big_orders.iloc[:, 0].records

   col  idx       size  price      fees  side
0    0    0  99.009901    1.0  0.990099     0
1    0    3  99.009901    2.0  1.980198     1
3.24 ms ± 560 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
print(orders.iloc[:, np.arange(portfolio.main_price.shape[1])].records)
%timeit big_orders.iloc[:, np.arange(portfolio.main_price.shape[1])].records

    col  idx        size  price      fees  side
0     0    0   99.009901    1.0  0.990099     0
1     0    3   99.009901    2.0  1.980198     1
2     1    0  198.019802    1.0  1.980198     0
3     1    1  198.019802    2.0  3.960396     1
4     1    2  129.399079    3.0  3.881972     0
5     1    3  129.399079    2.0  2.587982     1
6     1    4  253.673441    1.0  2.536734     0
7     2    1  148.514851    2.0  2.970297     0
8     2    2  148.514851    3.0  4.455446     1
9     2    3  218.360945    2.0  4.367219     0
10    2    4  218.360945    1.0  2.183609     1
3.48 ms ± 500 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Records

In [31]:
print(portfolio['a'].orders.records)
print(portfolio.orders['a'].records)
print(portfolio.orders.records)

%timeit big_portfolio.orders.records

   col  idx       size  price      fees  side
0    0    0  99.009901    1.0  0.990099     0
1    0    3  99.009901    2.0  1.980198     1
   col  idx       size  price      fees  side
0    0    0  99.009901    1.0  0.990099     0
1    0    3  99.009901    2.0  1.980198     1
    col  idx        size  price      fees  side
0     0    0   99.009901    1.0  0.990099     0
1     0    3   99.009901    2.0  1.980198     1
2     1    0  198.019802    1.0  1.980198     0
3     1    1  198.019802    2.0  3.960396     1
4     1    2  129.399079    3.0  3.881972     0
5     1    3  129.399079    2.0  2.587982     1
6     1    4  253.673441    1.0  2.536734     0
7     2    1  148.514851    2.0  2.970297     0
8     2    2  148.514851    3.0  4.455446     1
9     2    3  218.360945    2.0  4.367219     0
10    2    4  218.360945    1.0  2.183609     1
7.21 ms ± 646 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [32]:
print(portfolio['a'].trades.records)
print(portfolio.trades['a'].records)
print(portfolio.trades.records)

%timeit big_portfolio.trades.records

   col       size  entry_idx  entry_price  entry_fees  exit_idx  exit_price  \
0    0  99.009901          0          1.0    0.990099         3         2.0   

   exit_fees        pnl    return  status  position_idx  
0   1.980198  96.039604  0.960396       1             0  
   col       size  entry_idx  entry_price  entry_fees  exit_idx  exit_price  \
0    0  99.009901          0          1.0    0.990099         3         2.0   

   exit_fees        pnl    return  status  position_idx  
0   1.980198  96.039604  0.960396       1             0  
   col        size  entry_idx  entry_price  entry_fees  exit_idx  exit_price  \
0    0   99.009901          0          1.0    0.990099         3         2.0   
1    1  198.019802          0          1.0    1.980198         1         2.0   
2    1  129.399079          2          3.0    3.881972         3         2.0   
3    1  253.673441          4          1.0    2.536734         4         1.0   
4    2  148.514851          1          2.0    2.97

In [33]:
print(portfolio['a'].positions.records)
print(portfolio.positions['a'].records)
print(portfolio.positions.records)

%timeit big_portfolio.positions.records

   col       size  entry_idx  entry_price  entry_fees  exit_idx  exit_price  \
0    0  99.009901          0          1.0    0.990099         3         2.0   

   exit_fees        pnl    return  status  
0   1.980198  96.039604  0.960396       1  
   col       size  entry_idx  entry_price  entry_fees  exit_idx  exit_price  \
0    0  99.009901          0          1.0    0.990099         3         2.0   

   exit_fees        pnl    return  status  
0   1.980198  96.039604  0.960396       1  
   col        size  entry_idx  entry_price  entry_fees  exit_idx  exit_price  \
0    0   99.009901          0          1.0    0.990099         3         2.0   
1    1  198.019802          0          1.0    1.980198         1         2.0   
2    1  129.399079          2          3.0    3.881972         3         2.0   
3    1  253.673441          4          1.0    2.536734         4         1.0   
4    2  148.514851          1          2.0    2.970297         2         3.0   
5    2  218.360945        

In [34]:
print(portfolio['a'].drawdowns.records)
print(portfolio.drawdowns['a'].records)
print(portfolio.drawdowns.records)

%timeit big_portfolio.drawdowns.records

   col  start_idx  valley_idx  end_idx  status
0    0          2           3        4       0
   col  start_idx  valley_idx  end_idx  status
0    0          2           3        4       0
   col  start_idx  valley_idx  end_idx  status
0    0          2           3        4       0
1    1          1           4        4       0
2    2          0           1        2       1
3    2          2           4        4       0
15.6 ms ± 457 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Equity

In [35]:
portfolio.equity.values

array([[ 99.00990099, 198.01980198, 300.        ],
       [198.01980198, 392.07920792, 297.02970297],
       [297.02970297, 388.19723557, 441.08910891],
       [196.03960396, 256.21017547, 436.72189001],
       [196.03960396, 253.67344106, 216.17733556]])

In [36]:
print(portfolio['a'].equity)
print(portfolio.equity)

%timeit big_portfolio.equity

2020-01-01     99.009901
2020-01-02    198.019802
2020-01-03    297.029703
2020-01-04    196.039604
2020-01-05    196.039604
Name: a, dtype: float64
                     a           b           c
2020-01-01   99.009901  198.019802  300.000000
2020-01-02  198.019802  392.079208  297.029703
2020-01-03  297.029703  388.197236  441.089109
2020-01-04  196.039604  256.210175  436.721890
2020-01-05  196.039604  253.673441  216.177336
7.68 ms ± 435 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [37]:
print(portfolio['a'].final_equity)
print(portfolio.final_equity)

%timeit big_portfolio.final_equity

196.03960396039605
a    196.039604
b    253.673441
c    216.177336
dtype: float64
9.61 ms ± 1.3 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [38]:
print(portfolio['a'].total_profit)
print(portfolio.total_profit)

%timeit big_portfolio.total_profit

96.03960396039605
a    96.039604
b    53.673441
c   -83.822664
dtype: float64
14.9 ms ± 5.67 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Drawdown

In [39]:
print(portfolio['a'].drawdown)
print(portfolio.drawdown)

%timeit big_portfolio.drawdown

2020-01-01    0.00
2020-01-02    0.00
2020-01-03    0.00
2020-01-04   -0.34
2020-01-05   -0.34
Name: a, dtype: float64
               a         b         c
2020-01-01  0.00  0.000000  0.000000
2020-01-02  0.00  0.000000 -0.009901
2020-01-03  0.00 -0.009901  0.000000
2020-01-04 -0.34 -0.346535 -0.009901
2020-01-05 -0.34 -0.353005 -0.509901
22.2 ms ± 1.28 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [40]:
print(portfolio['a'].max_drawdown)
print(portfolio.max_drawdown)

%timeit big_portfolio.max_drawdown

-0.33999999999999997
a   -0.340000
b   -0.353005
c   -0.509901
dtype: float64
26.1 ms ± 1.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Returns

In [41]:
print(portfolio['a'].buy_and_hold_return)
print(portfolio.buy_and_hold_return)

%timeit big_portfolio.buy_and_hold_return

0.0
a    0.0
b    0.0
c    0.0
dtype: float64
11.2 ms ± 291 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [42]:
print(portfolio['a'].returns)
print(portfolio.returns)

%timeit big_portfolio.equity.pct_change()
%timeit big_portfolio.returns # numba helps a lot

2020-01-01   -0.009901
2020-01-02    1.000000
2020-01-03    0.500000
2020-01-04   -0.340000
2020-01-05    0.000000
Name: a, dtype: float64
                   a         b         c
2020-01-01 -0.009901 -0.009901  0.000000
2020-01-02  1.000000  0.980000 -0.009901
2020-01-03  0.500000 -0.009901  0.485000
2020-01-04 -0.340000 -0.340000 -0.009901
2020-01-05  0.000000 -0.009901 -0.505000
293 ms ± 5.63 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
15.2 ms ± 632 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [43]:
print(portfolio['a'].daily_returns)
print(portfolio.daily_returns)

%timeit big_portfolio.daily_returns

2020-01-01   -0.009901
2020-01-02    1.000000
2020-01-03    0.500000
2020-01-04   -0.340000
2020-01-05    0.000000
Name: a, dtype: float64
                   a         b         c
2020-01-01 -0.009901 -0.009901  0.000000
2020-01-02  1.000000  0.980000 -0.009901
2020-01-03  0.500000 -0.009901  0.485000
2020-01-04 -0.340000 -0.340000 -0.009901
2020-01-05  0.000000 -0.009901 -0.505000
17.2 ms ± 826 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [44]:
print(portfolio['a'].annual_returns)
print(portfolio.annual_returns)

%timeit big_portfolio.annual_returns

2020-01-01    0.960396
Freq: 252D, Name: a, dtype: float64
                   a         b         c
2020-01-01  0.960396  0.268367 -0.279409
23.1 ms ± 300 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [45]:
print(portfolio['a'].cumulative_returns)
print(portfolio.cumulative_returns)

%timeit big_portfolio.cumulative_returns

2020-01-01   -0.009901
2020-01-02    0.980198
2020-01-03    1.970297
2020-01-04    0.960396
2020-01-05    0.960396
Name: a, dtype: float64
                   a         b         c
2020-01-01 -0.009901 -0.009901  0.000000
2020-01-02  0.980198  0.960396 -0.009901
2020-01-03  1.970297  0.940986  0.470297
2020-01-04  0.960396  0.281051  0.455740
2020-01-05  0.960396  0.268367 -0.279409
29.5 ms ± 1.86 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [46]:
print(portfolio['a'].total_return)
print(portfolio.total_return)

%timeit big_portfolio.total_return

0.9603960396039604
a    0.960396
b    0.268367
c   -0.279409
dtype: float64
21.4 ms ± 503 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [47]:
print(portfolio['a'].annualized_return)
print(portfolio.annualized_return)

%timeit big_portfolio.annualized_return

542161095949729.56
a    5.421611e+14
b    1.597885e+05
c   -9.999999e-01
dtype: float64
20.6 ms ± 1.55 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [48]:
print(portfolio['a'].annualized_volatility)
print(portfolio.annualized_volatility)

%timeit big_portfolio.annualized_volatility

132.2191242654978
a    132.219124
b      7.944401
c      2.211017
dtype: float64
24.2 ms ± 3.55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [49]:
print(portfolio['a'].calmar_ratio)
print(portfolio.calmar_ratio)

%timeit big_portfolio.calmar_ratio

1594591458675675.2
a    1.594591e+15
b    4.526527e+05
c   -1.961165e+00
dtype: float64
32.1 ms ± 3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [50]:
print(portfolio['a'].omega_ratio)
print(portfolio.omega_ratio)

%timeit big_portfolio.omega_ratio

3.882163392568907
a    3.882163
b    2.119488
c    0.699507
dtype: float64
28.8 ms ± 2.16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [51]:
print(portfolio['a'].sharpe_ratio)
print(portfolio.sharpe_ratio)

%timeit big_portfolio.sharpe_ratio

6.656842846055576
a    6.656843
b    3.237371
c   -1.721495
dtype: float64
21.4 ms ± 2.89 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
print(portfolio['a'].downside_risk)
print(portfolio.downside_risk)

%timeit big_portfolio.downside_risk

3.2969960073204563
a    3.296996
b    4.621501
c    6.846689
dtype: float64
19.8 ms ± 1.41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [53]:
print(portfolio['a'].sortino_ratio)
print(portfolio.sortino_ratio)

%timeit big_portfolio.sortino_ratio

9.93783129438448
a     9.937831
b    -4.249924
c   -11.334824
dtype: float64
22.3 ms ± 1.11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
print(portfolio['a'].information_ratio)
print(portfolio.information_ratio)

%timeit big_portfolio.information_ratio

-1.9476532644416638
a   -1.947653
b   -1.784165
c   -3.033691
dtype: float64
21.6 ms ± 2.33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [55]:
print(portfolio['a'].beta)
print(portfolio.beta)

%timeit big_portfolio.beta

0.2936193619361937
a    0.293619
b    0.047129
c    0.318844
dtype: float64
24.8 ms ± 830 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [56]:
print(portfolio['a'].alpha)
print(portfolio.alpha)

%timeit big_portfolio.alpha

-1.0
a    -1.000000
b    92.486018
c    -1.000000
dtype: float64
29.3 ms ± 2.17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [57]:
print(portfolio['a'].tail_ratio)
print(portfolio.tail_ratio)

%timeit big_portfolio.tail_ratio

3.284908933217693
a    3.284909
b    2.854293
c    0.955712
dtype: float64
47.7 ms ± 2.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [58]:
print(portfolio['a'].value_at_risk)
print(portfolio.value_at_risk)

%timeit big_portfolio.value_at_risk

-0.32679603960396036
a   -0.326796
b   -0.313592
c   -0.445588
dtype: float64
30.5 ms ± 895 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [59]:
print(portfolio['a'].conditional_value_at_risk)
print(portfolio.conditional_value_at_risk)

%timeit big_portfolio.conditional_value_at_risk

-0.33999999999999997
a   -0.340
b   -0.340
c   -0.505
dtype: float64
29 ms ± 992 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [60]:
print(portfolio['a'].capture)
print(portfolio.capture)

%timeit big_portfolio.capture

8.411412319796107e-88
a     8.411412e-88
b    2.642871e-104
c   -4.724622e-116
dtype: float64
19.1 ms ± 973 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [61]:
print(portfolio['a'].up_capture)
print(portfolio.up_capture)

%timeit big_portfolio.up_capture

8.411412319796107e-88
a     8.411412e-88
b    2.642871e-104
c   -4.724622e-116
dtype: float64
22.2 ms ± 1.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [62]:
print(portfolio['a'].down_capture)
print(portfolio.down_capture)

%timeit big_portfolio.down_capture

nan
a   NaN
b   NaN
c   NaN
dtype: float64
15.2 ms ± 1.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Stats

In [63]:
print(portfolio['c'].stats)

%timeit big_portfolio[0].stats

Start                     2020-01-01 00:00:00
End                       2020-01-05 00:00:00
Duration                      5 days 00:00:00
Holding Duration [%]                       40
Total Profit                         -83.8227
Total Return [%]                     -27.9409
Buy & Hold Return [%]                       0
Max. Drawdown [%]                     50.9901
Avg. Drawdown [%]                     25.9901
Max. Drawdown Duration        2 days 00:00:00
Avg. Drawdown Duration        2 days 00:00:00
Num. Trades                                 2
Win Rate [%]                               50
Best Trade [%]                        47.0297
Worst Trade [%]                      -50.9901
Avg. Trade [%]                        -1.9802
Max. Trade Duration           1 days 00:00:00
Avg. Trade Duration           1 days 00:00:00
Expectancy                           -41.9113
SQN                                 -0.229023
Sharpe Ratio                         -1.72149
Sortino Ratio                     